In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow import keras
import cv2

%matplotlib inline
plt.rcParams['figure.figsize'] = 12, 8
%config InlineBackend.figure_format = 'retina'

In [ ]:
from keras.applications import VGG16
from keras.models import Sequential, load_model
from keras.layers import Input, Flatten, Dense, Dropout

vgg = VGG16(include_top=False, weights="imagenet")
vgg.trainable = False

dense = load_model("modele-VGG16") # ou 'modele-VGG16.h5'
modele = Sequential([
    Input((227, 227, 3)),
    vgg,
    dense
], name="complet")

In [ ]:
from os.path import join
NOM = "BowlPlace1Subject1"
VIDEO = join("VIDEOS", NOM + ".mp4")
BOITE = join("GT", NOM + "_2_bboxes.txt")

## Lecture des boites fournies
Lit le fichier contenant les informations de reconnaissance d'objets pour avoir un point de départ.

## Recherche de l'objet sur la prochaine image
À partir de la zone désignée dans l'image précédante, tente de retrouver l'objet dans la nouvelle image.

In [ ]:
video = cv2.VideoCapture(VIDEO)

while video.isOpened():
    ret, image = video.read()
    
    if ret == False:
        break
        
    # découper une boite
    # la tester
    # la déplacer un peu et réessayer -> méthode diamand ?
    
video.release()